Data loading and processing:

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

df = pd.read_csv('/Users/alibekabilmazhit/flirtio/flirt-generation/augmented-data/detection_train_augmented_with_replies.csv')

non_flirty_df = df[df['label'] == 0.0]
flirty_df = df[df['label'] == 1.0]

input_texts = list(df['input'].values)
output_texts = list(df['reply'].values)
labels = list(df['label'].values)

# Add START/END tokens to outputs
target_texts = ['<START> ' + text + ' <END>' for text in output_texts]

print(f"Total samples: {len(input_texts)}")
print(f"Non-flirty: {len(non_flirty_df)}, Flirty: {len(flirty_df)}")

Total samples: 5198
Non-flirty: 2598, Flirty: 2600


In [29]:
# Build vocabularies
input_tokenizer = Tokenizer(filters='', oov_token='<OOV>')
target_tokenizer = Tokenizer(filters='', oov_token='<OOV>')

input_tokenizer.fit_on_texts(input_texts)
target_tokenizer.fit_on_texts(target_texts)

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

print(f"Input vocab size: {input_vocab_size}, Target vocab size: {target_vocab_size}")

encoder_input_sequences = input_tokenizer.texts_to_sequences(input_texts)
decoder_input_sequences = target_tokenizer.texts_to_sequences(target_texts)

max_encoder_len = max([len(seq) for seq in encoder_input_sequences]) if encoder_input_sequences else 1
max_decoder_len = max([len(seq) for seq in decoder_input_sequences]) if decoder_input_sequences else 1

encoder_input_data = pad_sequences(encoder_input_sequences, maxlen=max_encoder_len, padding='post')
decoder_input_data = pad_sequences(decoder_input_sequences, maxlen=max_decoder_len, padding='post')

# Decoder targets (shifted by 1)
decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

# Convert labels to numpy array
label_data = np.array(labels, dtype=np.float32)

print(f"Encoder input shape: {encoder_input_data.shape}, Decoder input shape: {decoder_input_data.shape}")


Input vocab size: 5900, Target vocab size: 5137
Encoder input shape: (5198, 135), Decoder input shape: (5198, 18)


In [30]:
# MODEL BUILDING
embedding_dim = 128
latent_dim = 256

# ENCODER - includes label as input
encoder_inputs = Input(shape=(max_encoder_len,))
label_inputs = Input(shape=(1,))

encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)

# Concatenate label information with final state
label_dense = Dense(latent_dim)(label_inputs)
state_h = Concatenate()([state_h, label_dense])
state_h = Dense(latent_dim)(state_h)
state_c = Concatenate()([state_c, label_dense])
state_c = Dense(latent_dim)(state_c)

encoder_states = [state_h, state_c]

# DECODER
decoder_inputs = Input(shape=(max_decoder_len,))
decoder_embedding = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

# ATTENTION
attention = Attention()
context_vector = attention([decoder_outputs, encoder_outputs])

# CONCATENATE and OUTPUT
decoder_combined = Concatenate()([decoder_outputs, context_vector])
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined)

# MODEL
model = keras.Model([encoder_inputs, label_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 135)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 135, 128)  │    755,200 │ input_layer_12[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_13      │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 135,      │    394,240 │ embedding_4[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 256)       │        512 │ input_layer_13[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_14      │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 512)       │          0 │ lstm_4[0][1],     │
│ (Concatenate)       │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, 512)       │          0 │ lstm_4[0][2],     │
│ (Concatenate)       │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 18, 128)   │    657,536 │ input_layer_14[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256)       │    131,328 │ concatenate_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 256)       │    131,328 │ concatenate_9[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 18, 256), │    394,240 │ embedding_5[0][0… │
│                     │ (None, 256),      │            │ dense_9[0][0],    │
│                     │ (None, 256)]      │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_2         │ (None, 18, 256)   │          0 │ lstm_5[0][0],     │
│ (Attention)         │                   │            │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_10      │ (None, 18, 512)   │          0 │ lstm_5[0][0],     │
│ (Concatenate)       │                   │            │ attention_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 18, 5137)  │  2,635,281 │ concatenate_10[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,099,665 (19.45 MB)

 Trainable params: 5,099,665 (19.45 MB)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
# TRAINING
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    [encoder_input_data, label_data.reshape(-1, 1), decoder_input_data],
    decoder_target_data.reshape(decoder_target_data.shape[0], decoder_target_data.shape[1], 1),
    batch_size=32,
    epochs=20,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

# Save the trained model
model.save('flirty_response_model.h5')
print("\nModel saved as 'flirty_response_model.h5'")

Epoch 1/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 23s 170ms/step - accuracy: 0.4576 - loss: 4.0191 - val_accuracy: 0.4690 - val_loss: 3.7929
Epoch 2/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 21s 164ms/step - accuracy: 0.4929 - loss: 3.3804 - val_accuracy: 0.4766 - val_loss: 3.6221
Epoch 3/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 22s 170ms/step - accuracy: 0.5074 - loss: 3.1624 - val_accuracy: 0.4931 - val_loss: 3.4591
Epoch 4/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 23s 176ms/step - accuracy: 0.5246 - loss: 2.9612 - val_accuracy: 0.5029 - val_loss: 3.3460
Epoch 5/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 22s 171ms/step - accuracy: 0.5378 - loss: 2.7812 - val_accuracy: 0.5129 - val_loss: 3.2473
Epoch 6/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 23s 180ms/step - accuracy: 0.5514 - loss: 2.6189 - val_accuracy: 0.5222 - val_loss: 3.1625
Epoch 7/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 25s 189ms/step - accuracy: 0.5645 - loss: 2.4673 - val_accuracy: 0.5296 - val_loss: 3.0884
Epoch 8/20
130/130 ━━━━━━━━━━━━━━━━━━━━ 25s 192ms/step - accuracy: 0.5783 - loss: 2


Model saved as 'flirty_response_model.h5'


In [32]:
# Save tokenizers for inference
import pickle

with open('input_tokenizer.pkl', 'wb') as f:
    pickle.dump(input_tokenizer, f)
    
with open('target_tokenizer.pkl', 'wb') as f:
    pickle.dump(target_tokenizer, f)

print("Tokenizers saved as 'input_tokenizer.pkl' and 'target_tokenizer.pkl'")

Tokenizers saved as 'input_tokenizer.pkl' and 'target_tokenizer.pkl'


In [33]:
# 4. INFERENCE MODEL
# Encoder inference
encoder_model = keras.Model([encoder_inputs, label_inputs], [encoder_outputs, state_h, state_c])

# Decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_hidden_state_input = Input(shape=(max_encoder_len, latent_dim))

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]

context_vector = attention([decoder_outputs, decoder_hidden_state_input])
decoder_combined = Concatenate()([decoder_outputs, context_vector])
decoder_outputs = decoder_dense(decoder_combined)

decoder_model = keras.Model(
    [decoder_inputs, decoder_hidden_state_input] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [34]:
# GENERATION FUNCTION
def generate_response(input_text, flirty=False):
    """Generate response. If flirty=True, generate flirty response; else generate non-flirty response"""
    label = 1.0 if flirty else 0.0
    
    # Encode input
    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_encoder_len, padding='post')
    label_seq = np.array([[label]])
    
    encoder_output, h, c = encoder_model.predict([input_seq, label_seq], verbose=0)
    
    # Start token
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index.get('<START>', 1)
    
    decoded_sentence = ''
    
    for _ in range(max_decoder_len):
        output_tokens, h, c = decoder_model.predict(
            [target_seq, encoder_output, h, c], verbose=0
        )
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_tokenizer.index_word.get(sampled_token_index, '')
        
        if sampled_word == '<END>' or len(decoded_sentence.split()) > 20:
            break
        
        if sampled_word not in ['<START>', '<OOV>']:
            decoded_sentence += sampled_word + ' '
        
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
    
    return decoded_sentence.strip()

In [37]:
# TEST
print("\n=== GENERATION ===")
test_inputs = ["Hi, how are you?", "I am bored"]

for test in test_inputs:
    print(f"Input: {test}")
    print(f"Non-flirty response: {generate_response(test, flirty=False)}")
    print(f"Flirty response: {generate_response(test, flirty=True)}\n")


=== GENERATION ===
Input: Hi, how are you?
Non-flirty response: haha, yeah it good, thank for asking! <end>
Flirty response: feeling great, but i'm not too without making me ;) <end>

Input: I am bored
Non-flirty response: i prefer a great place to keep it in too! <end>
Flirty response: only if you want to keep me blush. afterwards 😉 <end>

